In [ ]:
from twython import Twython, TwythonStreamer, TwythonError, TwythonRateLimitError, TwythonAuthError
import jsonpickle
import time
import math

In [ ]:
CONSUMER_KEY = ''
CONSUMER_SECRET = ''

In [ ]:
twitter = Twython(CONSUMER_KEY, CONSUMER_SECRET, oauth_version = 2)
token = twitter.obtain_access_token()
twitter = Twython(CONSUMER_KEY, access_token = token)

In [ ]:
'''
    Parameters
    
    maxTweets:     number of downloaded tweets
    tweetPerQuery: max number of tweets per query API permits
    fName :        file to store tweets
    
    searchQuery:   search query
    resultType:    mixed, recent, popular 
    lang:          language of tweets
    sinceDate:     returns tweets created from the given date as YYYY-MM-DD
    untilDate:     returns tweets created before the given date as YYYY-MM-DD
    maxId:         returns tweets with ID less than
    placeId:       returns tweets created within placeID, US = '96683cc9126741d1'
    
'''
maxTweets = 10000
tweetPerQuery = 100
fName = 'tweets_kingsman.txt'

searchQuery = 'kingsman OR kingsmanmovie -filter:retweets'
resultType = 'mixed'
lang = 'en'
sinceDate = '2017-9-23'
untilDate = '2017-9-24'
maxId = None
placeId = '96683cc9126741d1' 

In [ ]:
tweetCount = 0
print("Downloading total {0} tweets".format(maxTweets))

with open(fName, 'w') as fw:
    while tweetCount < maxTweets:
        try:
            if tweetCount == 0:
                new_tweets = twitter.search(q = searchQuery,\
                                            result_type = resultType,\
                                            lang = lang,\
                                            count = tweetPerQuery,\
                                            since = sinceDate,\
                                            until = untilDate)['statuses']
            else:
                new_tweets = twitter.search(q = searchQuery,\
                                            result_type = resultType,\
                                            lang = lang,\
                                            count = tweetPerQuery,\
                                            since = sinceDate,\
                                            until = untilDate,\
                                            max_id = maxId - 1)['statuses']
            if not new_tweets:
                print("No more tweets found")
                break

            for tweet in new_tweets:
                fw.write(jsonpickle.encode(tweet, unpicklable = False) + '\n')

            tweetCount += len(new_tweets)
            maxId = new_tweets[-1]['id']
            print("Downloaded {0} tweets".format(tweetCount))
            
            callsLeft = float(twitter.get_lastfunction_header('x-rate-limit-remaining'))
            timeLeft =  float(twitter.get_lastfunction_header('x-rate-limit-reset')) - time.time()
            sleep_duration = math.ceil(timeLeft / callsLeft)
            time.sleep(sleep_duration)

        except TwythonError as e:
            print("Error: {0}".format(e.msg))
            break

print("Downloaded {0} tweets, Saved to {1}".format(tweetCount, fName))